## Testing assemble circuitry

In [ ]:
# # ===== SETUP CELL - Run this first =====
# import sys
# from pathlib import Path

# # Get repo root (assuming notebook is in notebooks/ folder)
# repo_root = Path.cwd().parents[0] if 'notebooks' in str(Path.cwd()) else Path.cwd()

# # Add to Python path
# if str(repo_root) not in sys.path:
#     sys.path.insert(0, str(repo_root))

# print(f"✓ Repo root: {repo_root}")
# print(f"✓ Python can now import from 'src/'")

✓ Repo root: c:\Users\Bahrs\Documents\_GitHub\gas-sensor_SWCNT_film-data-classification
✓ Python can now import from 'src/'


In [ ]:
import sys
from pathlib import Path

from thermocycling.pipeline.loading import load_gas_data  # pyright: ignore[reportMissingImports]
from thermocycling.pipeline.assemble import build_basic_dataset, full_dataset
from thermocycling.pipeline.cleaning import apply_manual_trim  # pyright: ignore[reportMissingImports]
from thermocycling.preprocessing.smoothing import dedrift, Exp_pd
#from data.paths import GAS_FILE_MAP

gas = 'NO2'
df = load_gas_data(gas)
df1 = build_basic_dataset(gas)
df2 = dedrift(df1, 201, Exp_pd, alpha = 0.0217)
df3 = full_dataset(Exp_pd, envelope_ind = [201], alpha = 0.0217)
print(df.shape, df1.shape, df2.shape, df3.shape)
print(df.columns[-6:].tolist(), df1.columns[-6:].tolist(), df2.columns[-6:].tolist(), df3.columns[-6:].tolist(), sep='\n')

(481998, 6) (1199, 408) (1199, 408) (3543, 408)
['V', 'I', 'MFC_target', 'flow_target_error', 'flow_carrier_error', 'meas_cycle']
['NO2', 'H2S', 'Acet', 'meas_cycle', 'gas_', 'class_']
['NO2', 'H2S', 'Acet', 'meas_cycle', 'gas_', 'class_']
['NO2', 'H2S', 'Acet', 'meas_cycle', 'gas_', 'class_']


In [ ]:
from thermocycling.preprocessing.train_test import build_sequences_for_df
#df3[df3["gas_"] == 'NO2'].loc[:,['NO2', 'H2S', 'Acet']]
data = build_sequences_for_df(df3, look_back=100)

In [ ]:
from thermocycling.preprocessing.train_test import create_time_series_folds
print("=" * 60)
print("CATBOOST FOLDS (2D)")
print("=" * 60)
cb_folds = create_time_series_folds(
    df3,
    model_type='catboost',
    feature_cols=402,
    target_cols=['NO2', 'H2S', 'Acet'],
    n_components=50,
    start_cycle=5
)

print(cb_folds.summary())

CATBOOST FOLDS (2D)
TimeSeriesCVSplitter: 4 folds

  Fold 0: train=2145, test=360, features=(50,)
  Fold 1: train=2505, test=360, features=(50,)
  Fold 2: train=2865, test=360, features=(50,)
  Fold 3: train=3225, test=318, features=(50,)


In [13]:
print("\n" + "=" * 60)
print("LSTM FOLDS (3D)")
print("=" * 60)
lstm_folds = create_time_series_folds(
    df3,
    model_type='lstm',
    feature_cols=402,
    target_cols=['NO2', 'H2S', 'Acet'],
    look_back=150,
    n_components=50,
    start_cycle=5,
    test_size=2,
)
print(lstm_folds.summary())


LSTM FOLDS (3D)
TimeSeriesCVSplitter: 3 folds

  Fold 0: train=1698, test=273, features=(150, 50)
  Fold 1: train=2058, test=273, features=(150, 50)
  Fold 2: train=2418, test=231, features=(150, 50)


In [14]:
# Test TensorFlow dataset conversion
print("\n" + "=" * 60)
print("TENSORFLOW DATASETS")
print("=" * 60)
tf_datasets = lstm_folds.to_tf_datasets(batch_size=32)
train_ds, test_ds = tf_datasets[0]
print(f"First fold - Train dataset: {train_ds}")
print(f"First fold - Test dataset: {test_ds}")


TENSORFLOW DATASETS
First fold - Train dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 50), dtype=tf.float64, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>
First fold - Test dataset: <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 50), dtype=tf.float64, name=None), TensorSpec(shape=(None, 3), dtype=tf.float64, name=None))>


In [ ]:
from thermocycling.preprocessing.train_test import TimeSeriesCVSplitter
# Test save/load
print("\n" + "=" * 60)
print("SAVE/LOAD TEST")
print("=" * 60)
cb_folds.save('test_folds')
loaded_folds = TimeSeriesCVSplitter.load('test_folds')
print(f"Successfully saved and loaded {len(loaded_folds)} folds")


SAVE/LOAD TEST
Successfully saved and loaded 4 folds


In [ ]:
def test_imports():
    """Test that all modules can be imported."""
    print("Testing imports...")
    
    try:
        from thermocycling.pipeline.loading import load_gas_data
        from thermocycling.pipeline.cleaning import apply_manual_trim
        from thermocycling.pipeline.assemble import build_basic_dataset, full_dataset
        from thermocycling.preprocessing.smoothing import Exp_pd, Savitzky_Golay, dedrift
        from thermocycling.preprocessing.train_test import create_time_series_folds
        from thermocycling.models.catboost_model import build_catboost_classifier, build_catboost_regressor
        from thermocycling.models.lstm_model import build_lstm
        from thermocycling.models.optuna_objectives import (
            LSTMRegressorObjective,
            CatBoostClassifierObjective
        )
        print("✓ All imports successful")
        return True
    except ImportError as e:
        print(f"✗ Import failed: {e}")
        return False


def test_data_loading():
    """Test data loading pipeline."""
    print("\nTesting data loading...")
    
    try:
        from thermocycling.pipeline.assemble import full_dataset
        from thermocycling.preprocessing.smoothing import Exp_pd
        
        df = full_dataset(
            dedrifting_func=Exp_pd,
            envelope_ind=[201],
            alpha=0.0217
        )
        
        assert df.shape[1] == 408, f"Expected 408 columns, got {df.shape[1]}"
        assert 'NO2' in df.columns, "Missing NO2 column"
        assert 'class_' in df.columns, "Missing class_ column"
        assert 'meas_cycle' in df.columns, "Missing meas_cycle column"
        
        print(f"✓ Data loaded successfully: {df.shape}")
        print(f"  Gases: {df['gas_'].unique()}")
        print(f"  Cycles: {df['meas_cycle'].min()} - {df['meas_cycle'].max()}")
        return True
    except Exception as e:
        print(f"✗ Data loading failed: {e}")
        return False


def test_cv_splitting():
    """Test CV splitting for both model types."""
    print("\nTesting CV splitting...")
    
    try:
        from thermocycling.pipeline.assemble import full_dataset
        from thermocycling.preprocessing.smoothing import Exp_pd
        from thermocycling.preprocessing.train_test import create_time_series_folds
        
        df = full_dataset(
            dedrifting_func=Exp_pd,
            envelope_ind=[201],
            alpha=0.0217
        )
        
        # Test CatBoost folds
        catboost_folds = create_time_series_folds(
            df,
            model_type='catboost',
            task_type='regressor',
            n_components=50,
            start_cycle=7,
            test_size=1
        )
        print(f"✓ CatBoost folds: {len(catboost_folds)} folds")
        
        # Test LSTM folds
        lstm_folds = create_time_series_folds(
            df,
            model_type='lstm',
            task_type='regressor',
            look_back=30,
            n_components=50,
            start_cycle=7,
            test_size=1
        )
        print(f"✓ LSTM folds: {len(lstm_folds)} folds")
        
        return True
    except Exception as e:
        print(f"✗ CV splitting failed: {e}")
        return False


def test_model_building():
    """Test model building."""
    print("\nTesting model building...")
    
    try:
        from thermocycling.models.catboost_model import build_catboost_regressor
        from thermocycling.models.lstm_model import build_lstm
        
        # Test CatBoost
        catboost_model = build_catboost_regressor(iterations=10, verbose=False)
        print("✓ CatBoost model built")
        
        # Test LSTM
        lstm_model = build_lstm(
            input_shape=(30, 50),  # (look_back, n_features)
            output_shape=3,  # 3 gas concentrations
            n_layers=2,
            n_units=32
        )
        lstm_model.build(input_shape=(None, 30, 50))  # Build the model first
        print("✓ LSTM model built")
        print(f"  LSTM parameters: {lstm_model.count_params():,}")
        
        return True
    except Exception as e:
        print(f"✗ Model building failed: {e}")
        return False


def test_config_loading():
    """Test config file loading."""
    print("\nTesting config loading...")
    
    try:
        import yaml
        
        config_files = [
            'configs/config_lstm_regression.yaml',
            'configs/config_catboost_classification.yaml',
            'configs/config_catboost_regression.yaml'
        ]
        
        for config_file in config_files:
            config_path = PROJECT_ROOT / config_file
            if not config_path.exists():
                print(f"⚠ Config file not found: {config_file}")
                continue
                
            with open(config_path, 'r') as f:
                config = yaml.safe_load(f)
            
            assert 'experiment' in config, f"Missing 'experiment' in {config_file}"
            assert 'data' in config, f"Missing 'data' in {config_file}"
            print(f"✓ Config loaded: {config_file}")
        
        return True
    except Exception as e:
        print(f"✗ Config loading failed: {e}")
        return False

In [17]:
print("="*60)
print("TESTING SETUP")
print("="*60)

tests = [
    ("Imports", test_imports),
    ("Data Loading", test_data_loading),
    ("CV Splitting", test_cv_splitting),
    ("Model Building", test_model_building),
    ("Config Loading", test_config_loading),
]

results = []
for name, test_func in tests:
    try:
        success = test_func()
        results.append((name, success))
    except Exception as e:
        print(f"✗ Test '{name}' crashed: {e}")
        results.append((name, False))

print("\n" + "="*60)
print("SUMMARY")
print("="*60)

for name, success in results:
    status = "✓ PASS" if success else "✗ FAIL"
    print(f"{status}: {name}")

all_passed = all(success for _, success in results)

if all_passed:
    print("\n🎉 All tests passed! Your setup is ready.")
else:
    print("\n❌ Some tests failed. Check the errors above.")

TESTING SETUP
Testing imports...


c:\Users\Bahrs\Documents\_GitHub\gas-sensor_SWCNT_film-data-classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ All imports successful

Testing data loading...
✓ Data loaded successfully: (3543, 408)
  Gases: ['NO2' 'H2S' 'Acet']
  Cycles: 0 - 9

Testing CV splitting...
✓ CatBoost folds: 2 folds
✓ LSTM folds: 2 folds

Testing model building...
✓ CatBoost model built
✓ LSTM model built
  LSTM parameters: 13,811

Testing config loading...
✓ Config loaded: configs/config_lstm_regression.yaml
✓ Config loaded: configs/config_catboost_classification.yaml
✓ Config loaded: configs/config_catboost_regression.yaml

SUMMARY
✓ PASS: Imports
✓ PASS: Data Loading
✓ PASS: CV Splitting
✓ PASS: Model Building
✓ PASS: Config Loading

🎉 All tests passed! Your setup is ready.


In [ ]:
import sys
import yaml
import argparse
import mlflow
from pathlib import Path

# Add src to path
#PROJECT_ROOT = Path(__file__).resolve().parents[1]  # PROJECT_ROOT = Path(__file__).resolve().parents[0]
#sys.path.insert(0, str(PROJECT_ROOT / "src"))

from thermocycling.pipeline.assemble import full_dataset
from thermocycling.preprocessing.smoothing import Exp_pd, Savitzky_Golay
from thermocycling.models.optuna_objectives import (
    run_lstm_regressor_optimization,
    run_lstm_classifier_optimization,
    run_catboost_classifier_optimization,
    run_catboost_regressor_optimization
)


DEDRIFT_METHODS = {
    'Exp_pd': Exp_pd,
    'Savitzky_Golay': Savitzky_Golay
}


def load_config(config_path: str) -> dict:
    """Load YAML configuration file."""
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    return config


def prepare_data(config: dict):
    """Prepare dataset according to config."""
    data_cfg = config['data']
    
    # Get dedrift method
    dedrift_method_name = data_cfg['dedrift_method']
    dedrift_func = DEDRIFT_METHODS[dedrift_method_name]
    dedrift_params = data_cfg['dedrift_params']
    
    # Load full dataset
    df = full_dataset(
        dedrifting_func=dedrift_func,
        **dedrift_params
    )
    
    print(f"✓ Loaded dataset: {df.shape}")
    print(f"  Gases: {df['gas_'].unique()}")
    print(f"  Measurement cycles: {df['meas_cycle'].min()} - {df['meas_cycle'].max()}")
    
    return df


def run_optimization(config: dict, df):
    """Run optimization based on config."""
    exp_cfg = config['experiment']
    opt_cfg = config['optimization']
    mlflow_cfg = config['mlflow']
    preproc_cfg = config['preprocessing']
    
    # Set MLflow tracking URI
    mlflow.set_tracking_uri(mlflow_cfg['tracking_uri'])
    
    # Prepare objective kwargs
    objective_kwargs = {
        'cv_start_cycle': preproc_cfg['cv_start_cycle'],
        'cv_test_size': preproc_cfg['cv_test_size'],
        'feature_cols': config['data']['feature_cols']
    }
    
    # Add model-specific parameters
    if exp_cfg['model_type'] == 'lstm':
        objective_kwargs.update({
            'look_back_range': preproc_cfg['look_back_range'],
            'n_components_range': preproc_cfg['n_components_range']
        })
        
        if exp_cfg['task'] == 'regression':
            objective_kwargs['target_cols'] = config['data']['target_cols']
    
    elif exp_cfg['model_type'] == 'catboost':
        objective_kwargs['n_components_range'] = preproc_cfg['n_components_range']
        
        if exp_cfg['task'] == 'regression':
            objective_kwargs['target_cols'] = config['data']['target_cols']
    
    # Select optimization function
    if exp_cfg['model_type'] == 'lstm' and exp_cfg['task'] == 'regression':
        optimize_func = run_lstm_regressor_optimization
    elif exp_cfg['model_type'] == 'lstm' and exp_cfg['task'] == 'classification':
        optimize_func = run_lstm_classifier_optimization
    elif exp_cfg['model_type'] == 'catboost' and exp_cfg['task'] == 'classification':
        optimize_func = run_catboost_classifier_optimization
    elif exp_cfg['model_type'] == 'catboost' and exp_cfg['task'] == 'regression':
        optimize_func = run_catboost_regressor_optimization
    else:
        raise ValueError(
            f"Unknown model_type/task combination: "
            f"{exp_cfg['model_type']}/{exp_cfg['task']}")
    
    # Run optimization
    print(f"\n🚀 Starting optimization: {exp_cfg['name']}")
    print(f"   Study: {exp_cfg['study_name']}")
    print(f"   Trials: {opt_cfg['n_trials']}")
    print(f"   Direction: {opt_cfg['direction']}")
    
    study = optimize_func(
        df,
        study_name=exp_cfg['study_name'],
        mlflow_experiment_name=exp_cfg['name'],
        n_trials=opt_cfg['n_trials'],
        timeout=opt_cfg['timeout'],
        direction=opt_cfg['direction'],
        objective_kwargs=objective_kwargs
    )
    
    print(f"\n✓ Optimization complete!")
    print(f"  Best value: {study.best_value:.6f}")
    print(f"  Best params: {study.best_params}")
    
    return study


def optimization_from_config_main():
    parser = argparse.ArgumentParser(
        description="Run hyperparameter optimization from config file"
    )
    parser.add_argument(
        'config',
        type=str,
        help='Path to YAML config file'
    )
    args = parser.parse_args()
    
    # Load config
    print(f"📄 Loading config: {args.config}")
    config = load_config(args.config)
    
    # Prepare data
    print(f"\n📊 Preparing data...")
    df = prepare_data(config)
    
    # Run optimization
    study = run_optimization(config, df)
    
    print(f"\n✅ Done! MLflow experiment: {config['experiment']['name']}")